# Chronicle SIEM API Discovery
This notebook is intended to aggregate basic functions to facilitate the use of the Chronicle SIEM API for exploratory and testing purposes (discovery).

To start using it, generate an access key through a service account.  Then, in Google Colab, add this key as a new secret called `CHRONICLE_SIEM_KEY`.

In [ ]:
#@title Session
#@markdown Start from here initializing the session. When the authentication
#@markdown is successful, use the `client` object to interact with Chronicle.

from json import loads

from google.auth.transport import requests
from google.oauth2 import service_account

from google.colab import userdata


AUTHORIZATION_SCOPES = [
  'https://www.googleapis.com/auth/chronicle-backstory',
  'https://www.googleapis.com/auth/malachite-ingestion'
]
CHRONICLE_API_BASE_URL = "https://backstory.googleapis.com"

def init_client(credentials, scopes=AUTHORIZATION_SCOPES):
  '''
  Obtains an authorized session using the provided credentials.
  Args:
    credentials (google.oauth2.service_account.Credentials): The service account credentials.
  Returns:
    requests.AuthorizedSession: An authorized session for making API calls.
  '''
  credentials = service_account.Credentials.from_service_account_info(
    credentials,
    scopes=AUTHORIZATION_SCOPES
  )
  return requests.AuthorizedSession(credentials)

try:
  client = init_client(loads(userdata.get('CHRONICLE_SIEM_KEY')))
  print('🟢 Session is up and `client` is ready 🚀')
except Exception as e:
  print(f'🔴 Authentication failed: {e}')

In [ ]:
#@title Reference Lists

from datetime import datetime
from json import dumps


def get_lists(client, params):
  '''
  Obtain all Reference Lists in Chronicle SIEM.
  Args:
    client: Session for making API calls.
  Returns:
    List of strings of all all reference lists' names.
  '''
  url = f'{CHRONICLE_API_BASE_URL}/v2/lists'
  res = client.request('GET', url, params=params)
  res.raise_for_status()
  return sorted([x['name'] for x in res.json()['lists']])

def get_lines(client, list_name):
  '''
  Fetches all lines in a reference list.
  Args:
    client: Session for making API calls.
    list_name: Name of the reference list.
  Returns:
    Full response from Chronicle API in JSON format.
  '''
  url = f'{CHRONICLE_API_BASE_URL}/v2/lists/{list_name}'
  res = client.request('GET', url)
  res.raise_for_status()
  return res.json()


dump = {
  'timestamp': datetime.now().isoformat(),
  'lists': list()
}
lists = get_lists(client, {'page_size':2000,'page_token':''})

for l in lists:
  dump['lists'].append(get_lines(client, l))

print(dumps(dump))

In [ ]:
#@title Feeds

from json import dumps

def get_feeds(client):
  '''
  Fetches all feeds in Chronicle SIEM.
  Args:
    client: Session for making API calls.
  Returns:
    Feeds in JSON format.
  '''
  url = f'{CHRONICLE_API_BASE_URL}/v1/feeds'
  res = client.request('GET', url)
  res.raise_for_status()
  return res.json()

def get_feed(client, id):
  '''
  Fetches an specific feed in Chronicle SIEM based on its ID.
  Args:
    client: Session for making API calls.
    id: ID of the feed (feed/{ID-HERE})
  Returns:
    Feed in JSON format.
  '''
  url = f'{CHRONICLE_API_BASE_URL}/v1/feeds/{id}'
  res = client.request('GET', url)
  res.raise_for_status()
  return res.json()


print(dumps(get_feeds(client)))

In [ ]:
#@title Rules


def get_rules(client, params):
  '''
  Fetches detection rules in Chronicle SIEM.
  Args:
    client: Session for making API calls.
    params: API call parameters.
  Returns:
    Rules in JSON format.
  '''
  url = f'{CHRONICLE_API_BASE_URL}/v2/detect/rules'
  res = client.request('GET', url, params=params)
  res.raise_for_status()
  return res.json()

print(get_rules(client, {'page_size':2000,'page_token':''}))